# RAG (Retrieval-Augmented Generation) 구현

이 노트북은 LangChain을 사용하여 RAG 시스템을 구현하는 방법을 보여줍니다. 유사도 기반 검색과 BM25 검색을 앙상블하여 더 나은 검색 결과를 얻는 방법을 다룹니다.

In [1]:
import os
import json
from typing import List, Dict, Any
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.ensemble import EnsembleRetriever
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

## 1. ChromaDB 로드 함수

벡터 데이터베이스를 로드하는 함수입니다.

In [2]:
def load_chroma_db(persist_directory: str):
    """
    ChromaDB를 로드합니다.
    
    Args:
        persist_directory: ChromaDB가 저장된 디렉토리 경로
        
    Returns:
        ChromaDB 인스턴스
    """
    # 임베딩 모델 생성
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    
    # ChromaDB 로드
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )
    
    return vectordb

## 2. 리트리버 생성 함수들

다양한 유형의 리트리버를 생성하는 함수들입니다.

In [3]:
def create_similarity_retriever(vectordb, k=4):
    """
    유사도 기반 리트리버를 생성합니다.
    
    Args:
        vectordb: 벡터 데이터베이스
        k: 검색할 문서 수
        
    Returns:
        유사도 기반 리트리버
    """
    return vectordb.as_retriever(search_type="similarity", search_kwargs={"k": k})

def create_bm25_retriever(documents, k=4):
    """
    BM25 리트리버를 생성합니다.
    
    Args:
        documents: 문서 리스트
        k: 검색할 문서 수
        
    Returns:
        BM25 리트리버
    """
    return BM25Retriever.from_documents(documents, k=k)

def create_ensemble_retriever(retrievers, weights=None):
    """
    앙상블 리트리버를 생성합니다.
    
    Args:
        retrievers: 리트리버 리스트
        weights: 각 리트리버의 가중치
        
    Returns:
        앙상블 리트리버
    """
    if weights is None:
        weights = [1/len(retrievers)] * len(retrievers)
    
    return EnsembleRetriever(
        retrievers=retrievers,
        weights=weights
    )

## 3. RAG 체인 생성 함수

리트리버와 언어 모델을 결합하여 RAG 체인을 생성하는 함수입니다.

In [4]:
def create_rag_chain(retriever, llm):
    """
    RAG 체인을 생성합니다.
    
    Args:
        retriever: 리트리버
        llm: 언어 모델
        
    Returns:
        RAG 체인
    """
    # 프롬프트 템플릿 생성
    prompt = ChatPromptTemplate.from_template("""
    다음 정보를 바탕으로 질문에 답변해주세요:
    
    <context>
    {context}
    </context>
    
    질문: {input}
    
    답변:
    """)
    
    # 문서 결합 체인 생성
    document_chain = create_stuff_documents_chain(llm, prompt)
    
    # 검색 체인 생성
    retrieval_chain = create_retrieval_chain(retriever, document_chain)
    
    return retrieval_chain

## 4. 메인 실행 코드

RAG 시스템을 실행하고 테스트하는 코드입니다.

In [5]:
# ChromaDB 경로
chroma_dir = "../chroma_db"

print(f"ChromaDB 로드 중: {chroma_dir}")
vectordb = load_chroma_db(chroma_dir)

# 모든 문서 가져오기 (BM25 리트리버용)
all_documents = vectordb.get()
documents = [Document(page_content=text, metadata=metadata) 
            for text, metadata in zip(all_documents['documents'], all_documents['metadatas'])]

print(f"총 {len(documents)}개의 문서를 로드했습니다.")

ChromaDB 로드 중: ../chroma_db


/tmp/ipykernel_2082623/2708055516.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


총 237개의 문서를 로드했습니다.


In [6]:
# 리트리버 생성
similarity_retriever = create_similarity_retriever(vectordb, k=3)
bm25_retriever = create_bm25_retriever(documents, k=3)

# 앙상블 리트리버 생성 (유사도 0.7, BM25 0.3 가중치)
ensemble_retriever = create_ensemble_retriever(
    retrievers=[similarity_retriever, bm25_retriever],
    weights=[0.7, 0.3]
)

# 언어 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# RAG 체인 생성
rag_chain = create_rag_chain(ensemble_retriever, llm)

## 5. 테스트 쿼리 실행

각 리트리버별 검색 결과를 확인하고 RAG 체인의 응답을 확인합니다.

In [7]:
# 테스트 쿼리
query = "Roo Code란 무엇인가요?"
print(f"\n테스트 쿼리: '{query}'")

# 각 리트리버별 검색 결과 확인
print("\n유사도 검색 결과:")
similarity_results = similarity_retriever.get_relevant_documents(query)
for i, doc in enumerate(similarity_results):
    print(f"\n결과 {i+1}:")
    print(f"내용: {doc.page_content[:150]}...")
    print(f"메타데이터: {doc.metadata}")

print("\nBM25 검색 결과:")
bm25_results = bm25_retriever.get_relevant_documents(query)
for i, doc in enumerate(bm25_results):
    print(f"\n결과 {i+1}:")
    print(f"내용: {doc.page_content[:150]}...")
    print(f"메타데이터: {doc.metadata}")

print("\n앙상블 검색 결과:")
ensemble_results = ensemble_retriever.get_relevant_documents(query)
for i, doc in enumerate(ensemble_results):
    print(f"\n결과 {i+1}:")
    print(f"내용: {doc.page_content[:150]}...")
    print(f"메타데이터: {doc.metadata}")


테스트 쿼리: 'Roo Code란 무엇인가요?'

유사도 검색 결과:


/tmp/ipykernel_2082623/354196260.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  similarity_results = similarity_retriever.get_relevant_documents(query)



결과 1:
내용: # Roo Code Docs  
Roo Code (formerly Roo Cline) is an AI-powered autonomous coding agent that lives in your editor. It helps you code faster and smart...
메타데이터: {'header1': 'Roo Code Docs', 'name': 'index.md', 'path': 'index.md'}

결과 2:
내용: ## General  
### What is Roo Code?  
Roo Code is an AI-powered autonomous coding agent that lives in your editor.  
### How does Roo Code work?  
Roo ...
메타데이터: {'header1': 'Frequently Asked Questions', 'header2': 'General', 'name': 'faq.md', 'path': 'faq.md'}

결과 3:
내용: ## What Can Roo Code Do?  
- 🚀 **Generate Code** from natural language descriptions
- 🔧 **Refactor & Debug** existing code
- 📝 **Write & Update** docu...
메타데이터: {'header1': 'Roo Code Docs', 'header2': 'What Can Roo Code Do?', 'name': 'index.md', 'path': 'index.md'}

BM25 검색 결과:

결과 1:
내용: ## Step 1: Open the Roo Code Panel  
If the Roo Code panel isn't already visible, click the Roo Code icon (<Codicon name="rocket" />) in the VS Code A...
메타데이터: {'header1': 'Starting You

In [8]:
# RAG 체인 실행
print("\nRAG 응답:")
response = rag_chain.invoke({"input": query})
print(response["answer"])


RAG 응답:
Roo Code는 코드 에디터 내에서 작동하는 AI 기반의 자율 코딩 에이전트입니다. 이 도구는 새로운 프로젝트를 시작하거나 기존 코드를 유지 관리하며, 새로운 기술을 배우는 데 도움을 줍니다. Roo Code는 대형 언어 모델(LLM)을 활용하여 사용자의 요청을 이해하고 이를 행동으로 변환합니다. 이를 통해 코드 생성, 리팩토링, 버그 수정, 문서 작성, 코드 설명, 코드베이스에 대한 질문 답변, 반복 작업 자동화, 새로운 파일 및 프로젝트 생성 등의 다양한 코딩 작업을 지원합니다.


## 6. 대화형 모드

사용자의 질문에 대해 RAG 시스템이 응답하는 대화형 모드입니다.

In [9]:
# 대화형 모드 (Jupyter에서는 이 셀을 여러 번 실행하여 질문할 수 있습니다)
user_query = input("질문을 입력하세요: ")
if user_query.lower() != 'q':
    response = rag_chain.invoke({"input": user_query})
    print("\n응답:")
    print(response["answer"])


응답:
MCP 설정 방법은 다음과 같습니다:

1. **Roo Code 설정 열기**: Gear 아이콘(<Codicon name="gear" />)을 클릭하여 Roo Code 설정을 엽니다.

2. **MCP 서버 사용 설정**:
   - **MCP 서버 완전 비활성화**: 
     - "Enable MCP Servers" 설정을 찾아 체크박스를 해제합니다. 
     - 이 옵션을 선택하면 Roo Code는 어떤 MCP 서버에도 연결되지 않으며, `use_mcp_tool`와 `access_mcp_resource` 도구를 사용할 수 없습니다.

   - **MCP 서버 생성 비활성화**:
     - "Enable MCP Server Creation" 설정을 찾아 체크박스를 해제합니다. 
     - 이 옵션을 비활성화하면 Roo Code는 새로운 MCP 서버를 생성하지 않습니다. 기존에 수동으로 구성된 서버는 여전히 사용할 수 있습니다.

3. **MCP 서버 구성 파일 편집**:
   - MCP 서버 구성은 `cline_mcp_settings.json` 파일에 저장됩니다. "Edit MCP Settings"를 클릭하여 접근하거나, VS Code 명령 팔레트에서 `Roo Code: Open MCP Config` 명령을 사용하여 열 수 있습니다.

4. **MCP 서버 상태 관리**:
   - 상단 내비게이션 바에서 서버 아이콘(<Codicon name="server" />)을 클릭하여 MCP 서버 연결 상태를 확인하고 관리할 수 있습니다.

이러한 단계를 통해 MCP 설정을 구성하고 조정할 수 있습니다.
